In [117]:
from __future__ import absolute_import, division, print_function

# Database set-up

1. take the '\*.json' files from the Flickr API query and put them in a PostgreSQL database. 

2. clean the database for duplicates from the image search.

## Command-line 

On the 'psql' command line, certain queries can be run:

* '\l' to list all databases the server is connected to
* '\c db_name' to connect to database with name 'db_name'
* '\dt+' to see tables within the database
* '\d table_name' to see details (e.g., schema) of table 'table_name'



In [118]:
import os
import json
import pandas as pd
import glob
import numpy as np
import sys


## Python packages - you may have to pip install sqlalchemy, sqlalchemy_utils, and psycopg2.
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2

In [119]:
#In Python: Define a database name (we're using a dataset on births, so I call it 
# birth_db), and your username and password used above. 
dbname = 'flickr_db'
#username = 'username'
#pswd = 'password'

In [120]:
## 'engine' is a connection to a database
## Here, we're using postgres, but sqlalchemy can connect to other things too.
#engine = create_engine('postgresql://%s:%s@localhost/%s'%(username,pswd,dbname))
engine = create_engine('postgresql://@localhost/%s'%(dbname))

In [121]:
print(engine.url)
# Replace localhost with IP address if accessing a remote server

postgresql://@localhost/flickr_db


In [122]:
## create a database (if it doesn't exist)
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

True


In [123]:
# Connect to an existing database
conn = psycopg2.connect("dbname=flickr_db")


In [124]:
# Open a cursor to perform database operations
cur = conn.cursor()


In [125]:
# Get schema for db from query response
in_folder = os.path.join('..','data','photo_search','Napa')
files = glob.glob(os.path.join(in_folder,'*.json'))
total_all_id = []
for idx,tmp_file in enumerate(files):
    with open(tmp_file, 'r') as f:
        data = json.load(f)
    n_total = int(data['photos']['total'])
    if n_total == 0:
        pass
    else:
        photo = data['photos']['photo'][0]
        print(photo)
        flickr_keys = photo.keys()
        break


{u'geo_is_public': 1, u'place_id': u'Fcg9CH9TVrIdknMo', u'owner': u'53487315@N00', u'id': u'435803691', u'title': u'flowers at Chateau St. Jean', u'woeid': u'2486809', u'geo_is_friend': 0, u'geo_is_contact': 0, u'datetaken': u'2004-05-30 17:49:42', u'farm': 1, u'secret': u'0f57b1452d', u'latitude': u'38.427924', u'accuracy': u'16', u'isfamily': 0, u'ispublic': 1, u'tags': u'california flower gardens wine sonoma chateaustjean flowersfoliage pss:opd=1085935782', u'isfriend': 0, u'geo_is_family': 0, u'dateupload': u'1085935782', u'width_m': u'500', u'datetakenunknown': 0, u'datetakengranularity': u'0', u'longitude': u'-122.547935', u'server': u'176', u'url_m': u'https://farm1.staticflickr.com/176/435803691_0f57b1452d.jpg', u'context': 0, u'height_m': u'333'}


In [126]:
# Show keys
for flickr_key in flickr_keys:
    print(flickr_key)

geo_is_public
place_id
owner
id
title
woeid
geo_is_friend
geo_is_contact
datetaken
farm
secret
latitude
accuracy
isfamily
ispublic
tags
isfriend
geo_is_family
dateupload
width_m
datetakenunknown
datetakengranularity
longitude
server
url_m
context
height_m


In [127]:
# Turn into dictionary with 'flickr_key' being the key and POSTGRES column type the value.
postgre_schema = {str(tmp_key): '' for tmp_key in flickr_keys}
print(postgre_schema)

{'geo_is_public': '', 'place_id': '', 'owner': '', 'id': '', 'title': '', 'woeid': '', 'geo_is_friend': '', 'geo_is_contact': '', 'datetaken': '', 'isfriend': '', 'secret': '', 'latitude': '', 'accuracy': '', 'isfamily': '', 'ispublic': '', 'tags': '', 'farm': '', 'geo_is_family': '', 'dateupload': '', 'width_m': '', 'datetakenunknown': '', 'datetakengranularity': '', 'longitude': '', 'server': '', 'url_m': '', 'context': '', 'height_m': ''}


In [128]:
# Hardcode format
# check here for varchar vs text
# http://stackoverflow.com/questions/4848964/postgresql-difference-between-text-and-varchar-character-varying

format_dict = {
    'geo_is_public': 'integer' , 
    'place_id': 'text',
    'owner': 'text',
    'id': 'bigint',
    'title': 'text',
    'woeid': 'integer',
    'geo_is_friend': 'integer',
    'geo_is_contact': 'integer',
    'datetaken': 'timestamp',
    'farm': 'integer',
    'secret': 'text',
    'latitude': 'real',
    'accuracy': 'integer',
    'isfamily': 'integer',
    'ispublic': 'integer',
    'tags': 'text',
    'isfriend': 'integer',
    'geo_is_family': 'integer',
    'dateupload': 'integer',
    'width_m': 'integer',
    'datetakenunknown': 'integer',
    'datetakengranularity': 'integer',
    'longitude': 'real',
    'server': 'integer',
    'url_m': 'text',
    'context': 'integer',
    'height_m': 'integer'}

In [129]:
# Merge postgres schema with format dict
for tmp_key in postgre_schema.keys():
    tmp_format = format_dict[tmp_key]
    postgre_schema[tmp_key] = tmp_format
print(postgre_schema)

{'geo_is_public': 'integer', 'place_id': 'text', 'owner': 'text', 'id': 'bigint', 'title': 'text', 'woeid': 'integer', 'geo_is_friend': 'integer', 'geo_is_contact': 'integer', 'datetaken': 'timestamp', 'isfriend': 'integer', 'secret': 'text', 'latitude': 'real', 'accuracy': 'integer', 'isfamily': 'integer', 'ispublic': 'integer', 'tags': 'text', 'farm': 'integer', 'geo_is_family': 'integer', 'dateupload': 'integer', 'width_m': 'integer', 'datetakenunknown': 'integer', 'datetakengranularity': 'integer', 'longitude': 'real', 'server': 'integer', 'url_m': 'text', 'context': 'integer', 'height_m': 'integer'}


In [130]:
# Add 'county' and Primary key
postgre_schema['county'] = 'text'
postgre_schema['pk'] = 'serial PRIMARY KEY'

In [131]:
# Turn into ordered list with fixed length columns first, leading with primary key
# http://stackoverflow.com/questions/12604744/does-the-order-of-columns-in-a-postgres-table-impact-performance
postgre_schema_list = []
# Primary key first
postgre_schema_list += ([(k, postgre_schema[k]) 
                         for k in postgre_schema.keys() if k == 'pk'])
postgre_schema_list += ([(k, postgre_schema[k]) 
                         for k in postgre_schema.keys() if postgre_schema[k] == 'integer'])
postgre_schema_list += ([(k, postgre_schema[k]) 
                         for k in postgre_schema.keys() if postgre_schema[k] == 'bigint'])
postgre_schema_list += ([(k, postgre_schema[k]) 
                         for k in postgre_schema.keys() if postgre_schema[k] == 'real'])
postgre_schema_list += ([(k, postgre_schema[k]) 
                         for k in postgre_schema.keys() if postgre_schema[k] == 'timestamp'])
postgre_schema_list += ([(k, postgre_schema[k]) 
                         for k in postgre_schema.keys() if postgre_schema[k] == 'text'])
# Check that everything is still there
if len(postgre_schema_list) == len(postgre_schema.keys()):
    print('All is well!')
else:
    print('You forgot something!')


All is well!


In [132]:
# Build 'CREATE TABLE; string
tmp_str = ''
for idx, column in enumerate(postgre_schema_list):
    tmp_str = tmp_str+column[0]+' '+column[1]
    if idx < len(postgre_schema_list)-1:
        tmp_str = tmp_str+', '
str_create_table = "CREATE TABLE flickr_all ("+tmp_str+");"
print(str_create_table)

CREATE TABLE flickr_all (pk serial PRIMARY KEY, geo_is_public integer, woeid integer, geo_is_friend integer, geo_is_contact integer, isfriend integer, accuracy integer, isfamily integer, ispublic integer, farm integer, geo_is_family integer, dateupload integer, width_m integer, datetakenunknown integer, datetakengranularity integer, server integer, context integer, height_m integer, id bigint, latitude real, longitude real, datetaken timestamp, place_id text, county text, owner text, title text, secret text, tags text, url_m text);


In [133]:
# Execute a command: this creates a new table
cur.execute(str_create_table)


In [134]:
# Make the changes to the database persistent
conn.commit()

In [135]:
# Create 'INSERT' string
def create_insert_string_from_dict(indict,county):
    """
    From input dictionary extract key, value pairs 
    and build separate strings for SQL 'INSERT'. 
    Watch out for order in schema.
    """
    # Rearrange key/values pairs according to 'postgre_schema_list'
    keys_str = ''
    values_str = ''
    for idx,psl in enumerate(postgre_schema_list):
        if psl[0] not in indict.keys():
            pass
        elif psl[0] == 'pk':
            pass
        elif psl[0] == 'county':
            keys_str += psl[0]+","
            values_str += "'"+county+"',"
        elif psl[1] == 'timestamp':
            keys_str += psl[0]+","
            time_str = str(indict[psl[0]])
            if '0000' in time_str:
                #print(time_str)
                time_str = '1800-01-01 00:00:00'
            values_str += "TIMESTAMP '"+time_str+"',"
        else:
            keys_str += psl[0]+","
            try:
                tmp_str = str(indict[psl[0]])
            except UnicodeEncodeError:
                tmp_str = indict[psl[0]].encode(errors='ignore')
            if "'" in tmp_str:
                tmp_str = tmp_str.replace("'","")
            values_str += "'"+tmp_str+"',"
    # remove trailing comma
    keys_str = keys_str[:-1]
    values_str = values_str[:-1]
    return keys_str, values_str


In [160]:
# Now that I know how to inject data, let's do the same for all other counties
counties = ['Alameda','Contra Costa','Marin','San Francisco',
            'San Mateo','Santa Clara','Santa Cruz','Solano','Sonoma']
# But to be conservative, let's do it one county at a time
counties = ['Sonoma']
fail_list = []
error_list = []
for county in counties:
    print(county)
    in_folder = os.path.join('..','data','photo_search',county)
    files = glob.glob(os.path.join(in_folder,'*.json'))
    for idx,tmp_file in enumerate(files):
        with open(tmp_file, 'r') as f:
            data = json.load(f)
        n_total = data['photos']['total']
        if n_total == 0:
            pass
        else:
            for photo in data['photos']['photo']:
                keys_str, values_str = create_insert_string_from_dict(photo,county)
                insert_str = "INSERT INTO flickr_all ("+keys_str+") VALUES ("+values_str+");"
                cur.execute(insert_str)


Sonoma


In [161]:
# Inspect fail files and error messages
print(len(fail_list))


0


In [162]:
# Make the changes to the database persistent
conn.commit()

# Notes

* Alameda: 22 entries had 'date_taken' of '0000-01-01 00:00:00' or even '0000-00-00 00:00:00', which produced an error from the database because it could not be converted to a datetime object. I changed it to '1800-01-01 00:00:00' while creating the INSERT string to avoid this error.

* Contra Costa: Same issue as above for 95 entries. 

* Marin: 4 issues with 'date_taken' (as above)

* San Francisco: 1040 date errors

* San Mateo: 13 date errors

* Santa Clara: 16 date errors

* Santa Cruz: 5 date errors

* Solano: No INSERT errors

* Sonoma: No INSERT errors

In [179]:
# Count total number of id's
counties = ['Alameda','Contra Costa','Marin','San Francisco',
            'San Mateo','Santa Clara','Santa Cruz','Solano','Sonoma']
# But to be conservative, let's do it one county at a time
n_id = 0
for county in counties:
    print(county)
    in_folder = os.path.join('..','data','photo_search',county)
    files = glob.glob(os.path.join(in_folder,'*.json'))
    for idx,tmp_file in enumerate(files):
        with open(tmp_file, 'r') as f:
            data = json.load(f)
        n_total = data['photos']['total']
        if n_total == 0:
            pass
        else:
            for photo in data['photos']['photo']:
                tmp = photo['id']
                n_id += 1

Alameda
Contra Costa
Marin
San Francisco
San Mateo
Santa Clara
Santa Cruz
Solano
Sonoma


In [180]:
# Total number of id's in json files
print(n_id)

8310560


In [175]:
# Check that all id's have been commited
cur.execute("SELECT (id) FROM flickr_all;")

In [176]:
import copy
rows = cur.fetchall()
final_rows = copy.copy(rows)


In [181]:
n_sql = len(final_rows)

In [182]:
if n_sql == n_id:
    print('All is well!')

All is well!


In [178]:
# Unique entries in id column
print(len(set(final_rows)))

4913276


In [183]:
# But for some reason, county was not commited
cur.execute("SELECT (county) FROM flickr_all;")
rows = cur.fetchall()
final_rows = copy.copy(rows)
print(final_rows[0:10])

[(None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,), (None,)]


In [184]:
# Close communication with the database
cur.close()
conn.close()

# Querying the database

In [ ]:
# Check what is in the database
cur.execute("SELECT (pk) FROM flickr_all WHERE dateupload > 1472688000;")
#cur.execute("SELECT * FROM flickr_all WHERE pk < 20;")

In [ ]:
rows = cur.fetchall()
#print(rows)
for idx,row in enumerate(rows):
    if idx < 10:
        print(row)

In [ ]:
# Go directly into pands?
# sql_query = """
# SELECT * FROM flickr_all;
# """
# birth_data_from_sql = pd.read_sql_query(sql_query,con)

# birth_data_from_sql.head()

# More commands for interacting with the database

In [114]:
# rollback transaction that created errors
conn.rollback()

In [115]:
# Delete entire table from db
cur.execute("DROP TABLE flickr_all;")

In [116]:
# And commit the deletion
conn.commit()